# Automate Predicting Loan Eligibility Process 
### Benchmark Top ML Algorithms

In [1]:
# Different preprocessing methods can affect the accuracy score. Replacing NaN with mean/ most occuring value/

In [2]:
# Import neccessary library packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
RSEED = 35

In [3]:
# Import the train dataset
url = "https://raw.githubusercontent.com/subashgandyer/datasets/main/loan_train.csv"

In [4]:
# Load the training datasets
training_DF = pd.read_csv(url)
training_DF.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


### Preprocess Raw Dataset

In [5]:
training_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [6]:
# Replace "Married", "Education", "Self_Employed" text in training dataset with values 
married = {"Yes": 1, "No": 0}
education = {"Graduate": 1, "Not Graduate": 0}
self_employed = {"Yes": 1, "No": 0}
loan_status = {"Y": 1, "N": 0}

training_DF['Married'].replace(married, inplace=True)
training_DF['Education'].replace(education, inplace=True)
training_DF['Self_Employed'].replace(self_employed, inplace=True)
training_DF['Loan_Status'].replace(loan_status, inplace=True)
training_DF

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,0.0,0,1,0.0,5849,0.0,NaN,360.0,1.0,Urban,1
1,LP001003,Male,1.0,1,1,0.0,4583,1508.0,128.0,360.0,1.0,Rural,0
2,LP001005,Male,1.0,0,1,1.0,3000,0.0,66.0,360.0,1.0,Urban,1
3,LP001006,Male,1.0,0,0,0.0,2583,2358.0,120.0,360.0,1.0,Urban,1
4,LP001008,Male,0.0,0,1,0.0,6000,0.0,141.0,360.0,1.0,Urban,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,0.0,0,1,0.0,2900,0.0,71.0,360.0,1.0,Rural,1
610,LP002979,Male,1.0,3+,1,0.0,4106,0.0,40.0,180.0,1.0,Rural,1
611,LP002983,Male,1.0,1,1,0.0,8072,240.0,253.0,360.0,1.0,Urban,1
612,LP002984,Male,1.0,2,1,0.0,7583,0.0,187.0,360.0,1.0,Urban,1


In [7]:
# Split "Gender" into subsets in training dataset
gender = pd.get_dummies(training_DF['Gender'])

# Split "Property_Area" into subsets in training dataset
property_area = pd.get_dummies(training_DF['Property_Area'])

In [8]:
# Drop the original column and replace with the subsets
training_DF.drop(['Gender','Property_Area','Loan_ID'],axis=1,inplace=True)

In [9]:
training_DF = pd.concat([training_DF, gender, property_area],axis=1)
training_DF

,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,Female,Male,Rural,Semiurban,Urban
0,0.0,0,1,0.0,5849,0.0,NaN,360.0,1.0,1,0,1,0,0,1
1,1.0,1,1,0.0,4583,1508.0,128.0,360.0,1.0,0,0,1,1,0,0
2,1.0,0,1,1.0,3000,0.0,66.0,360.0,1.0,1,0,1,0,0,1
3,1.0,0,0,0.0,2583,2358.0,120.0,360.0,1.0,1,0,1,0,0,1
4,0.0,0,1,0.0,6000,0.0,141.0,360.0,1.0,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,0.0,0,1,0.0,2900,0.0,71.0,360.0,1.0,1,1,0,1,0,0
610,1.0,3+,1,0.0,4106,0.0,40.0,180.0,1.0,1,0,1,1,0,0
611,1.0,1,1,0.0,8072,240.0,253.0,360.0,1.0,1,0,1,0,0,1
612,1.0,2,1,0.0,7583,0.0,187.0,360.0,1.0,1,0,1,0,0,1


In [10]:
# Replace the NaN in training dataset with the mean value

In [11]:
training_DF.describe()

,Married,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,Female,Male,Rural,Semiurban,Urban
count,611.000000,614.000000,582.000000,614.000000,614.000000,592.000000,600.00000,564.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000
mean,0.651391,0.781759,0.140893,5403.459283,1621.245798,146.412162,342.00000,0.842199,0.687296,0.182410,0.796417,0.291531,0.379479,0.328990
std,0.476920,0.413389,0.348211,6109.041673,2926.248369,85.587325,65.12041,0.364878,0.463973,0.386497,0.402991,0.454838,0.485653,0.470229
min,0.000000,0.000000,0.000000,150.000000,0.000000,9.000000,12.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,0.000000,2877.500000,0.000000,100.000000,360.00000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
50%,1.000000,1.000000,0.000000,3812.500000,1188.500000,128.000000,360.00000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,1.000000,1.000000,0.000000,5795.000000,2297.250000,168.000000,360.00000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,81000.000000,41667.000000,700.000000,480.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [12]:
print(len(training_DF))

614


In [13]:
# Replace "3+" to 3
training_DF['Dependents'].replace({'0':0, '1':1, '2':2, '3+': 3}, inplace=True)

In [14]:
for col in training_DF.columns: 
    nnan = sum(np.isnan(training_DF[col]))
    print(col, nnan)

Married 3
Dependents 15
Education 0
Self_Employed 32
ApplicantIncome 0
CoapplicantIncome 0
LoanAmount 22
Loan_Amount_Term 14
Credit_History 50
Loan_Status 0
Female 0
Male 0
Rural 0
Semiurban 0
Urban 0


In [15]:
training_DF['Married'].fillna(training_DF['Married'].mean(), inplace=True)

In [16]:
training_DF['Dependents'].fillna(training_DF['Dependents'].mean(), inplace=True)

In [17]:
training_DF['Self_Employed'].fillna(training_DF['Self_Employed'].mean(), inplace=True)

In [18]:
training_DF['LoanAmount'].fillna(training_DF['LoanAmount'].mean(), inplace=True)

In [19]:
training_DF['Loan_Amount_Term'].fillna(training_DF['Loan_Amount_Term'].mean(), inplace=True)

In [20]:
training_DF['Credit_History'].fillna(training_DF['Credit_History'].mean(), inplace=True)

In [21]:
# Sanity Check (Checking for NaN and texts)
for col in training_DF.columns: 
    nnan = sum(np.isnan(training_DF[col]))
    print(col, nnan)

Married 0
Dependents 0
Education 0
Self_Employed 0
ApplicantIncome 0
CoapplicantIncome 0
LoanAmount 0
Loan_Amount_Term 0
Credit_History 0
Loan_Status 0
Female 0
Male 0
Rural 0
Semiurban 0
Urban 0


In [22]:
training_DF.head()

,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,Female,Male,Rural,Semiurban,Urban
0,0.0,0.0,1,0.0,5849,0.0,146.412162,360.0,1.0,1,0,1,0,0,1
1,1.0,1.0,1,0.0,4583,1508.0,128.000000,360.0,1.0,0,0,1,1,0,0
2,1.0,0.0,1,1.0,3000,0.0,66.000000,360.0,1.0,1,0,1,0,0,1
3,1.0,0.0,0,0.0,2583,2358.0,120.000000,360.0,1.0,1,0,1,0,0,1
4,0.0,0.0,1,0.0,6000,0.0,141.000000,360.0,1.0,1,0,1,0,0,1


In [23]:
# Split Clean Dataset into X and y
X = training_DF.drop(['Loan_Status', 'Female'], axis=1)
X

,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Male,Rural,Semiurban,Urban
0,0.0,0.0,1,0.0,5849,0.0,146.412162,360.0,1.0,1,0,0,1
1,1.0,1.0,1,0.0,4583,1508.0,128.000000,360.0,1.0,1,1,0,0
2,1.0,0.0,1,1.0,3000,0.0,66.000000,360.0,1.0,1,0,0,1
3,1.0,0.0,0,0.0,2583,2358.0,120.000000,360.0,1.0,1,0,0,1
4,0.0,0.0,1,0.0,6000,0.0,141.000000,360.0,1.0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,0.0,0.0,1,0.0,2900,0.0,71.000000,360.0,1.0,0,1,0,0
610,1.0,3.0,1,0.0,4106,0.0,40.000000,180.0,1.0,1,1,0,0
611,1.0,1.0,1,0.0,8072,240.0,253.000000,360.0,1.0,1,0,0,1
612,1.0,2.0,1,0.0,7583,0.0,187.000000,360.0,1.0,1,0,0,1


In [24]:
y = training_DF['Loan_Status']
y

0      1
1      0
2      1
3      1
4      1
      ..
609    1
610    1
611    1
612    1
613    0
Name: Loan_Status, Length: 614, dtype: int64

In [25]:
X.shape, y.shape

((614, 13), (614,))

In [26]:
# Split dataset into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                y, 
                                                test_size=0.2,
                                                random_state=3
                                               )
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((491, 13), (123, 13), (491,), (123,))

### Import the models
- Decision Tree
- KNN
- Logistic Regression
- SVM
- Random Forest
- Gaussian Naive Bayes

In [27]:
# Import necessary libraries
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [28]:
# Create the base models
dt = DecisionTreeClassifier(random_state=RSEED)
knn = KNeighborsClassifier()
lr = LogisticRegression(solver='liblinear', random_state=RSEED)
svm = SVC(random_state=RSEED)
rf = RandomForestClassifier(random_state=RSEED)
bayes = GaussianNB()

models = {'dt': dt,
          'knn': knn,
          'lr': lr,
          'svm': svm,
          'rf': rf,
          'bayes': bayes
         }

In [29]:
# Accuracy of the base models
for name, model in models.items():
    # fit the model on the training dataset
    model.fit(X_train, y_train)
    # make a prediction on the test dataset
    model_preds = model.predict(X_test)
    # evaluate the predictions
    score = accuracy_score(y_test, model_preds)
    # report the score
    print(f"Model = {model} has {score} accuracy")

Model = DecisionTreeClassifier(random_state=35) has 0.6829268292682927 accuracy
Model = KNeighborsClassifier() has 0.6747967479674797 accuracy
Model = LogisticRegression(random_state=35, solver='liblinear') has 0.8536585365853658 accuracy
Model = SVC(random_state=35) has 0.7723577235772358 accuracy
Model = RandomForestClassifier(random_state=35) has 0.8373983739837398 accuracy
Model = GaussianNB() has 0.8455284552845529 accuracy


### Use GridSearchCV for finding the best model with the best hyperparameters

In [30]:
# Create the parameter grid
param_grid_dt = {'max_leaf_nodes': list(range(2, 100)), 
                   'min_samples_split': [1, 2, 3, 4],
                   'max_features': ['auto', 'sqrt', 'log2'],
                   'criterion': ['gini', 'entropy']
                    }
param_grid_knn = {'n_neighbors': list(range(10, 50)),
                  'weights': ['uniform', 'distance'],
                  'algorithm': ['auto', 'brute']
                 }
param_grid_lr = {'C': [0.01, 0.1, 1, 10, 100],
                 'penalty': ['l1', 'l2'],
                 'solver': ['newton-cg', 'lbfgs', 'liblinear'],
                 'max_iter': [50, 100, 400]
                }
param_grid_svm = {'C': [0.1, 1, 10], 
                  'gamma': ['scale', 'auto'], 
                  'kernel': ['linear', 'rbf']
                 } 
param_grid_rf = {'n_estimators' : [300, 500, 800, 1000],
                 'max_depth' : [20, 30, 50, 80, 100],
                 'min_samples_split' : [2, 5, 10, 15, 100]
                }
param_grid_bayes = {'var_smoothing': np.logspace(0,-9, num=100)}

In [31]:
def matrix_accuracy(matrix):
    return np.diag(matrix).sum() / matrix.sum()

In [32]:
exp_configs = {
    'DT': {
        'grid_config': param_grid_dt,
        'model': DecisionTreeClassifier()
    },
    'KNN' : {
        'grid_config': param_grid_knn,
        'model': KNeighborsClassifier()
    },
    'LR': {
        'grid_config': param_grid_lr,
        'model': LogisticRegression()
    },
    'SVM': {
        'grid_config': param_grid_svm,
        'model': SVC()
    },
    'RF': {
        'grid_config': param_grid_rf,
        'model': RandomForestClassifier()
    },
    'BAYES': {
        'grid_config': param_grid_bayes,
        'model': GaussianNB()
    }
}

results = {}
for exp_name in exp_configs:
    exp_config = exp_configs[exp_name]
    # Initialize grid 
    grid = GridSearchCV(exp_config['model'], exp_config['grid_config'], scoring='accuracy', verbose=3)
    # Train all models in exp
    grid.fit(X_train, y_train)
    # Evaluate best
    preds = grid.predict(X_test)
    comatrix = confusion_matrix(y_test, preds)
    acc = matrix_accuracy(comatrix)
    
    result = {
        'best_param': grid.best_params_,
        'best_estimator' : grid.best_estimator_,
        'best_acc': acc
    }
    results[exp_name] = result

Fitting 5 folds for each of 2352 candidates, totalling 11760 fits
[CV] criterion=gini, max_features=auto, max_leaf_nodes=2, min_samples_split=1 
[CV]  criterion=gini, max_features=auto, max_leaf_nodes=2, min_samples_split=1, score=nan, total=   0.0s
[CV] criterion=gini, max_features=auto, max_leaf_nodes=2, min_samples_split=1 
[CV]  criterion=gini, max_features=auto, max_leaf_nodes=2, min_samples_split=1, score=nan, total=   0.0s
[CV] criterion=gini, max_features=auto, max_leaf_nodes=2, min_samples_split=1 
[CV]  criterion=gini, max_features=auto, max_leaf_nodes=2, min_samples_split=1, score=nan, total=   0.0s
[CV] criterion=gini, max_features=auto, max_leaf_nodes=2, min_samples_split=1 
[CV]  criterion=gini, max_features=auto, max_leaf_nodes=2, min_samples_split=1, score=nan, total=   0.0s
[CV] criterion=gini, max_features=auto, max_leaf_nodes=2, min_samples_split=1 
[CV]  criterion=gini, max_features=auto, max_leaf_nodes=2, min_samples_split=1, score=nan, total=   0.0s
[CV] criterion

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV]  criterion=gini, max_features=auto, max_leaf_nodes=3, min_samples_split=3, score=0.663, total=   0.0s
[CV] criterion=gini, max_features=auto, max_leaf_nodes=3, min_samples_split=3 
[CV]  criterion=gini, max_features=auto, max_leaf_nodes=3, min_samples_split=3, score=0.816, total=   0.0s
[CV] criterion=gini, max_features=auto, max_leaf_nodes=3, min_samples_split=3 
[CV]  criterion=gini, max_features=auto, max_leaf_nodes=3, min_samples_split=3, score=0.663, total=   0.0s
[CV] criterion=gini, max_features=auto, max_leaf_nodes=3, min_samples_split=4 
[CV]  criterion=gini, max_features=auto, max_leaf_nodes=3, min_samples_split=4, score=0.788, total=   0.0s
[CV] criterion=gini, max_features=auto, max_leaf_nodes=3, min_samples_split=4 
[CV]  criterion=gini, max_features=auto, max_leaf_nodes=3, min_samples_split=4, score=0.673, total=   0.0s
[CV] criterion=gini, max_features=auto, max_leaf_nodes=3, min_samples_split=4 
[CV]  criterion=gini, max_features=auto, max_leaf_nodes=3, min_samples

[CV]  criterion=gini, max_features=auto, max_leaf_nodes=6, min_samples_split=4, score=0.646, total=   0.0s
[CV] criterion=gini, max_features=auto, max_leaf_nodes=6, min_samples_split=4 
[CV]  criterion=gini, max_features=auto, max_leaf_nodes=6, min_samples_split=4, score=0.806, total=   0.0s
[CV] criterion=gini, max_features=auto, max_leaf_nodes=6, min_samples_split=4 
[CV]  criterion=gini, max_features=auto, max_leaf_nodes=6, min_samples_split=4, score=0.776, total=   0.0s
[CV] criterion=gini, max_features=auto, max_leaf_nodes=6, min_samples_split=4 
[CV]  criterion=gini, max_features=auto, max_leaf_nodes=6, min_samples_split=4, score=0.745, total=   0.0s
[CV] criterion=gini, max_features=auto, max_leaf_nodes=6, min_samples_split=4 
[CV]  criterion=gini, max_features=auto, max_leaf_nodes=6, min_samples_split=4, score=0.735, total=   0.0s
[CV] criterion=gini, max_features=auto, max_leaf_nodes=7, min_samples_split=1 
[CV]  criterion=gini, max_features=auto, max_leaf_nodes=7, min_samples

[CV]  criterion=gini, max_features=auto, max_leaf_nodes=10, min_samples_split=1, score=nan, total=   0.0s
[CV] criterion=gini, max_features=auto, max_leaf_nodes=10, min_samples_split=1 
[CV]  criterion=gini, max_features=auto, max_leaf_nodes=10, min_samples_split=1, score=nan, total=   0.0s
[CV] criterion=gini, max_features=auto, max_leaf_nodes=10, min_samples_split=1 
[CV]  criterion=gini, max_features=auto, max_leaf_nodes=10, min_samples_split=1, score=nan, total=   0.0s
[CV] criterion=gini, max_features=auto, max_leaf_nodes=10, min_samples_split=1 
[CV]  criterion=gini, max_features=auto, max_leaf_nodes=10, min_samples_split=1, score=nan, total=   0.0s
[CV] criterion=gini, max_features=auto, max_leaf_nodes=10, min_samples_split=1 
[CV]  criterion=gini, max_features=auto, max_leaf_nodes=10, min_samples_split=1, score=nan, total=   0.0s
[CV] criterion=gini, max_features=auto, max_leaf_nodes=10, min_samples_split=2 
[CV]  criterion=gini, max_features=auto, max_leaf_nodes=10, min_sample

...

[CV] . var_smoothing=3.511191734215127e-05, score=0.704, total=   0.0s
[CV] var_smoothing=3.511191734215127e-05 .............................
[CV] . var_smoothing=3.511191734215127e-05, score=0.643, total=   0.0s
[CV] var_smoothing=2.848035868435799e-05 .............................
[CV] . var_smoothing=2.848035868435799e-05, score=0.646, total=   0.0s
[CV] var_smoothing=2.848035868435799e-05 .............................
[CV] . var_smoothing=2.848035868435799e-05, score=0.643, total=   0.0s
[CV] var_smoothing=2.848035868435799e-05 .............................
[CV] . var_smoothing=2.848035868435799e-05, score=0.622, total=   0.0s
[CV] var_smoothing=2.848035868435799e-05 .............................
[CV] . var_smoothing=2.848035868435799e-05, score=0.704, total=   0.0s
[CV] var_smoothing=2.848035868435799e-05 .............................
[CV] . var_smoothing=2.848035868435799e-05, score=0.643, total=   0.0s
[CV] var_smoothing=2.310129700083158e-05 .............................
[CV] .

[CV] . var_smoothing=1.519911082952933e-06, score=0.643, total=   0.0s
[CV] var_smoothing=1.519911082952933e-06 .............................
[CV] . var_smoothing=1.519911082952933e-06, score=0.622, total=   0.0s
[CV] var_smoothing=1.519911082952933e-06 .............................
[CV] . var_smoothing=1.519911082952933e-06, score=0.704, total=   0.0s
[CV] var_smoothing=1.519911082952933e-06 .............................
[CV] . var_smoothing=1.519911082952933e-06, score=0.633, total=   0.0s
[CV] var_smoothing=1.232846739442066e-06 .............................
[CV] . var_smoothing=1.232846739442066e-06, score=0.646, total=   0.0s
[CV] var_smoothing=1.232846739442066e-06 .............................
[CV] . var_smoothing=1.232846739442066e-06, score=0.643, total=   0.0s
[CV] var_smoothing=1.232846739442066e-06 .............................
[CV] . var_smoothing=1.232846739442066e-06, score=0.622, total=   0.0s
[CV] var_smoothing=1.232846739442066e-06 .............................
[CV] .

[CV] . var_smoothing=8.111308307896873e-08, score=0.694, total=   0.0s
[CV] var_smoothing=8.111308307896873e-08 .............................
[CV] . var_smoothing=8.111308307896873e-08, score=0.643, total=   0.0s
[CV] var_smoothing=6.579332246575682e-08 .............................
[CV] . var_smoothing=6.579332246575682e-08, score=0.646, total=   0.0s
[CV] var_smoothing=6.579332246575682e-08 .............................
[CV] . var_smoothing=6.579332246575682e-08, score=0.643, total=   0.0s
[CV] var_smoothing=6.579332246575682e-08 .............................
[CV] . var_smoothing=6.579332246575682e-08, score=0.622, total=   0.0s
[CV] var_smoothing=6.579332246575682e-08 .............................
[CV] . var_smoothing=6.579332246575682e-08, score=0.694, total=   0.0s
[CV] var_smoothing=6.579332246575682e-08 .............................
[CV] . var_smoothing=6.579332246575682e-08, score=0.653, total=   0.0s
[CV] var_smoothing=5.336699231206302e-08 .............................
[CV] .

[CV] . var_smoothing=4.328761281083061e-09, score=0.786, total=   0.0s
[CV] var_smoothing=4.328761281083061e-09 .............................
[CV] . var_smoothing=4.328761281083061e-09, score=0.735, total=   0.0s
[CV] var_smoothing=4.328761281083061e-09 .............................
[CV] . var_smoothing=4.328761281083061e-09, score=0.837, total=   0.0s
[CV] var_smoothing=4.328761281083061e-09 .............................
[CV] . var_smoothing=4.328761281083061e-09, score=0.776, total=   0.0s
[CV] var_smoothing=3.5111917342151273e-09 ............................
[CV]  var_smoothing=3.5111917342151273e-09, score=0.808, total=   0.0s
[CV] var_smoothing=3.5111917342151273e-09 ............................
[CV]  var_smoothing=3.5111917342151273e-09, score=0.786, total=   0.0s
[CV] var_smoothing=3.5111917342151273e-09 ............................
[CV]  var_smoothing=3.5111917342151273e-09, score=0.745, total=   0.0s
[CV] var_smoothing=3.5111917342151273e-09 ............................
[CV]  

[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    2.9s finished


In [33]:
results

{'DT': {'best_param': {'criterion': 'entropy',
   'max_features': 'sqrt',
   'max_leaf_nodes': 6,
   'min_samples_split': 3},
  'best_estimator': DecisionTreeClassifier(criterion='entropy', max_features='sqrt',
                         max_leaf_nodes=6, min_samples_split=3),
  'best_acc': 0.7886178861788617},
 'KNN': {'best_param': {'algorithm': 'auto',
   'n_neighbors': 42,
   'weights': 'distance'},
  'best_estimator': KNeighborsClassifier(n_neighbors=42, weights='distance'),
  'best_acc': 0.7642276422764228},
 'LR': {'best_param': {'C': 1,
   'max_iter': 50,
   'penalty': 'l2',
   'solver': 'liblinear'},
  'best_estimator': LogisticRegression(C=1, max_iter=50, solver='liblinear'),
  'best_acc': 0.8536585365853658},
 'SVM': {'best_param': {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'},
  'best_estimator': SVC(C=0.1, kernel='linear'),
  'best_acc': 0.8455284552845529},
 'RF': {'best_param': {'max_depth': 20,
   'min_samples_split': 100,
   'n_estimators': 500},
  'best_estimator': R

In [34]:
train_acc = (results['RF']['best_estimator'].predict(X_train) == y_train).mean()
test_acc = (results['RF']['best_estimator'].predict(X_test) == y_test).mean()
print(train_acc, test_acc)

0.7983706720977597 0.8617886178861789


In [35]:
def load_data_by_url(url, for_test=False):
    training_DF = pd.read_csv(url)
    training_DF.head()
    
    # Replace "Married", "Education", "Self_Employed" text in training dataset with values 
    married = {"Yes": 1, "No": 0}
    education = {"Graduate": 1, "Not Graduate": 0}
    self_employed = {"Yes": 1, "No": 0}
    loan_status = {"Y": 1, "N": 0}
    
    training_DF['Married'].replace(married, inplace=True)
    training_DF['Education'].replace(education, inplace=True)
    training_DF['Self_Employed'].replace(self_employed, inplace=True)
    if not for_test:
        training_DF['Loan_Status'].replace(loan_status, inplace=True)
    
    # Split "Gender" into subsets in training dataset
    gender = pd.get_dummies(training_DF['Gender'])

    # Split "Property_Area" into subsets in training dataset
    property_area = pd.get_dummies(training_DF['Property_Area'])
    
    # Drop the original column and replace with the subsets
    training_DF.drop(['Gender','Property_Area','Loan_ID'],axis=1,inplace=True)
    
    training_DF = pd.concat([training_DF, gender, property_area],axis=1)
    
    # Replace "3+" to 3
    training_DF['Dependents'].replace({'0':0, '1':1, '2':2, '3+': 3}, inplace=True)
    
    cols = [
        'Married',
        'Dependents',
        'Self_Employed',
        'LoanAmount',
        'Loan_Amount_Term',
        'Credit_History',
    ]
    for col in cols:
        training_DF[col].fillna(training_DF[col].mean(), inplace=True)
    
    if not for_test:
        X = training_DF.drop(['Loan_Status', 'Female'], axis=1)
        y = training_DF['Loan_Status']
        return X, y 
    else:
        X = training_DF.drop(['Female'], axis=1)
    

## Verify the data loading function can reproduce previous results

In [36]:
new_train_x, new_train_y = load_data_by_url("https://raw.githubusercontent.com/subashgandyer/datasets/main/loan_train.csv")
_, x_test_new , _, y_test_new = train_test_split(new_train_x, 
                                                new_train_y, 
                                                test_size=0.2,
                                                random_state=3
                                               )
test_acc_new = (results['RF']['best_estimator'].predict(x_test_new) == y_test_new).mean()
print(test_acc_new, test_acc)

0.8617886178861789 0.8617886178861789


## Run the code below to get prediction on test.csv

In [37]:
final_test_x = load_data_by_url("https://raw.githubusercontent.com/subashgandyer/datasets/main/loan_test.csv", for_test=True)
final_test_preds = results['RF']['best_estimator'].predict(x_test_new)
final_test_preds

array([1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1])